<a href="https://colab.research.google.com/github/omnidox/local_detectron/blob/main/Webcam_BoundingBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


From Detectron

In [ ]:
!pip install fiftyone
!pip install fiftyone-db-ubuntu2204

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

Migrating database to v0.21.6


INFO:fiftyone.migrations.runner:Migrating database to v0.21.6


In [ ]:
!python -m pip install pyyaml==5.1

# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44091 sha256=5119ab121057157705523cb47b46f7577750192c67add29a5f2459763363d826
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2023.8.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
distributed 2023.8.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.7.2 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatible.

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.modeling import build_model, build_roi_heads
from detectron2.config import get_cfg

In [ ]:
# Continuation Training but with a scheduler for optimization


from detectron2.engine import DefaultTrainer

cfg = get_cfg()

# Use the configuration for Faster R-CNN
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("fiftyone_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2


# Use the checkpoint for Faster R-CNN
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.004
cfg.SOLVER.MAX_ITER = 40000
cfg.SOLVER.STEPS = []
cfg.SOLVER.GAMMA = 0.1
# cfg.SOLVER.WARMUP_ITERS = 2000
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.OUTPUT_DIR = '/content/drive/MyDrive/faster_rcnn/Weights_1'

cfg.SOLVER.CHECKPOINT_PERIOD = 2400

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
import torch

# Load the model checkpoint
checkpoint = torch.load(os.path.join("/content/drive/MyDrive/faster_rcnn/Weights_1/model_0077999.pth"))

# Extract the number of iterations
iterations = checkpoint['iteration']

print(f"The model was trained for {iterations} iterations.")


The model was trained for 77999 iterations.


In [ ]:

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5


cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/faster_rcnn/Weights_1/model_0077999.pth")  # path to the model we just trained
predictor = DefaultPredictor(cfg)


[08/23 18:21:29 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/faster_rcnn/Weights_1/model_0077999.pth ...


In [ ]:
# import fiftyone as fo

export_dir = "/content/drive/MyDrive/Alpha_dataset_02"

# Import the dataset
imported_dataset = fo.Dataset.from_dir(
    dataset_dir=export_dir,
    dataset_type=fo.types.FiftyOneDataset,
    name="imported_dataset_name"  # Choose a name for the imported dataset
)

Importing samples...


INFO:fiftyone.utils.data.importers:Importing samples...


 100% |███████████████| 7339/7339 [564.2ms elapsed, 0s remaining, 13.0K samples/s]     


INFO:eta.core.utils: 100% |███████████████| 7339/7339 [564.2ms elapsed, 0s remaining, 13.0K samples/s]     


Import complete


INFO:fiftyone.utils.data.importers:Import complete


In [ ]:
# Get the list of classes for the "ground_truth" field
classes = imported_dataset.distinct("ground_truth.detections.label")

# Print the classes
print(classes)

['Adhesive tape', 'Apple', 'Artichoke', 'Ball', 'Balloon', 'Banana', 'Beer', 'Bell pepper', 'Belt', 'Book', 'Boot', 'Bottle', 'Bowl', 'Box', 'Broccoli', 'Cake', 'Calculator', 'Camera', 'Candle', 'Carrot', 'Cheese', 'Chopsticks', 'Clock', 'Clothing', 'Coin', 'Computer mouse', 'Cookie', 'Cucumber', 'Dice', 'Doughnut', 'Drinking straw', 'Flower', 'Flying disc', 'Garden Asparagus', 'Glove', 'Grape', 'Handbag', 'Hat', 'High heels', 'Juice', 'Knife', 'Lemon', 'Mobile phone', 'Muffin', 'Mug', 'Mushroom', 'Orange', 'Oyster', 'Pastry', 'Peach', 'Pear', 'Pen', 'Pencil case', 'Plastic bag', 'Platter', 'Popcorn', 'Potato', 'Pretzel', 'Radish', 'Remote control', 'Roller skates', 'Ruler', 'Saucer', 'Scarf', 'Scissors', 'Shirt', 'Shorts', 'Skirt', 'Sock', 'Spoon', 'Stapler', 'Strawberry', 'Swimwear', 'Tablet computer', 'Teddy bear', 'Tie', 'Towel', 'Watch', 'Wine', 'Zucchini']


In [ ]:
# classes = ['Apple', 'Orange', 'Peach', 'Strawberry', 'Grape', 'Pear', 'Lemon', 'Banana',
# 'Bottle', 'Beer', 'Juice', 'Wine',
# 'Carrot', 'Bell pepper', 'Cucumber', 'Broccoli', 'Garden Asparagus', 'Zucchini', 'Radish', 'Artichoke', 'Mushroom', 'Potato',
# 'Pretzel', 'Popcorn', 'Muffin', 'Cheese', 'Cake', 'Cookie', 'Pastry', 'Doughnut',
# 'Pen', 'Adhesive tape', 'Pencil case', 'Stapler', 'Scissors', 'Ruler',
# 'Ball', 'Balloon', 'Dice', 'Flying disc', 'Teddy bear',
# 'Platter', 'Bowl', 'Knife', 'Spoon', 'Saucer', 'Chopsticks', 'Drinking straw', 'Mug',
# 'Glove', 'Belt', 'Sock', 'Tie', 'Watch', 'Computer mouse', 'Coin', 'Calculator', 'Box', 'Boot', 'Towel', 'Shorts', 'Swimwear',
# 'Shirt', 'Clock', 'Hat', 'Scarf', 'Roller skates', 'Skirt', 'Mobile phone',
# 'Plastic bag', 'High heels', 'Handbag', 'Clothing', 'Oyster', 'Tablet computer', 'Book', 'Flower', 'Candle', 'Camera', 'Remote control']


In [ ]:
num_classes = len(classes)
print("Number of classes: ", num_classes)

Number of classes:  80


In [ ]:
# Empty Polygon Error correction for fiftyone.
from detectron2.structures import BoxMode

def get_fiftyone_dicts(samples, classes):
    samples.compute_metadata()

    dataset_dicts = []
    for sample in samples.select_fields(["id", "filepath", "metadata", "ground_truth"]):
        height = sample.metadata["height"]
        width = sample.metadata["width"]
        record = {}
        record["file_name"] = sample.filepath
        record["image_id"] = sample.id
        record["height"] = height
        record["width"] = width

        objs = []
        for det in sample.ground_truth.detections:
            if det.label in classes:
                tlx, tly, w, h = det.bounding_box
                bbox = [int(tlx*width), int(tly*height), int(w*width), int(h*height)]
                fo_poly = det.to_polyline()
                if fo_poly.points:
                    poly = [(x*width, y*height) for x, y in fo_poly.points[0]]
                    poly = [p for x in poly for p in x]
                    if len(poly) >= 6:  # Check if the coordinates are sufficient to form a polygon
                        obj = {
                            "bbox": bbox,
                            "bbox_mode": BoxMode.XYWH_ABS,
                            "segmentation": [poly],
                            "category_id": classes.index(det.label),
                        }
                        objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts


In [ ]:
# Helps clear catalog data

from detectron2.data import DatasetCatalog, MetadataCatalog
DatasetCatalog.clear()
MetadataCatalog.clear()


In [ ]:
# Sets up Catalog Data for default fiftyone structured files


for d in ["train", "val"]:
    MetadataCatalog.get("fiftyone_" + d).set(thing_classes=classes)

metadata = MetadataCatalog.get("fiftyone_train")

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=a98c179e9639b662fcac42062dfd1644206f9974125c76f792d31c3c6ee3810d
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
from flask import Flask, Response
import cv2
from pyngrok import conf, ngrok
import requests
import json


In [ ]:
import requests
import json
import gzip
import base64

# define your Flask-ngrok url here
url = "http://montclair-object-detector.ngrok.app/"

def classify_objects(objects_list):
    for object_info in objects_list:
        class_name = object_info["class_name"]
        object_id = object_info["object_id"]
        box = object_info["box"]

        scores = {
            category: score_dict[category].get(class_name, 0) * 100
            for category in score_dict
        }

        if all(score == 0 for score in scores.values()):
            scores["Misc"] = 100

        print(f"Class Name: {class_name}")
        print(f"Object ID: {object_id}")
        print(f"Box: {box}")
        for category, score in scores.items():
            if score != 0:
                print(f"{category}: {score}%")
        print("\n")

        # Sending POST request to the Flask server
        # payload = {
        #     "class_name": class_name,
        #     "object_id": object_id,
        #     "box": box,
        #     "segmentation": segmentation
        #     # .cpu().numpy().tolist()  # Convert to list here
        # }


        # # Convert the numpy array to bytes
        # segmentation_bytes = segmentation.cpu().numpy().tobytes()

        # # Encode the bytes to a base64 string
        # segmentation_b64 = base64.b64encode(segmentation_bytes).decode('utf-8')

        # payload = {
        #     "class_name": class_name,
        #     "object_id": object_id,
        #     "box": box,
        #     "segmentation": segmentation_b64,
        #     "shape": segmentation.shape
        # }

        # json_data = json.dumps(payload)
        # compressed_data = gzip.compress(bytes(json_data, 'utf-8'))
        # headers = {'Content-Encoding': 'gzip'}

        # response = requests.post(url, data=compressed_data, headers=headers)
        # print("Response: ", response.json())

    print("---------------------------------------------------")


In [ ]:
# import requests
# import json

# # define your Flask-ngrok url here
# url = "http://montclair-object-detector.ngrok.app/"

# def classify_objects(objects_list):
#     for object_info in objects_list:
#         class_name = object_info["class_name"]
#         object_id = object_info["object_id"]
#         segmentation = object_info["segmentation"]
#         box = object_info["box"]

#         scores = {
#             category: score_dict[category].get(class_name, 0) * 100
#             for category in score_dict
#         }

#         if all(score == 0 for score in scores.values()):
#             scores["Misc"] = 100

#         print(f"Class Name: {class_name}")
#         print(f"Object ID: {object_id}")
#         print(f"Box: {box}")
#         print(f"Segmentation: {segmentation}")
#         for category, score in scores.items():
#             if score != 0:
#                 print(f"{category}: {score}%")
#         print("\n")

#         # Sending POST request to the Flask server
#         payload = {
#             "class_name": class_name,
#             "object_id": object_id,
#             "box": box,
#             "segmentation": segmentation.cpu().numpy().tolist()  # Convert to list here
#         }

#         response = requests.post(url, json=payload)
#         print("Response: ", response.json())

#     print("---------------------------------------------------")


In [ ]:
def generate_objects_list(outputs, cfg):
    instances = outputs["instances"]

    metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
    class_catalog = metadata.thing_classes

    return [{
        "class_name": class_catalog[instances.pred_classes[idx]],
        "object_id": idx,
        "box": coordinates.tolist()
    } for idx, coordinates in enumerate(instances.pred_boxes.tensor)]


score_dict = {
    "Fruit": {
        'Apple': 0.98, 'Orange': 0.46, 'Peach': 0.62, 'Strawberry': 0.92,
        'Grape': 0.96, 'Pear': 0.91, 'Banana': 0.95, 'Carrot': 0.88,
        'Pepper': 0.82, 'Cucumber': 0.91, 'Zucchini': 0.97,
        'Radish': 0.76, 'Artichoke': 0.7, 'Cheese': 0.75
    },
    "Drink": {
        'Bottle': 0.66, 'Beer': 0.82, 'Juice': 0.73, 'Wine': 0.84,
        'Mug': 0.96, 'Spoon': 0.46, 'Saucer': 0.46, 'Hat': 0.46,
    },
    "Vegetable": {
        'Strawberry': 0.7, 'Banana': 0.76, 'Carrot': 0.97, 'Pepper': 0.7,
        'Cucumber': 0.88, 'Broccoli': 0.82, 'Garden Asparagus': 0.91,
        'Zucchini': 0.89, 'Artichoke': 0.75, 'Mushroom': 0.76,
        'Potato': 0.91, 'Cheese': 0.7
    },
    "Snacks": {'Apple': 0.69, 'Banana': 0.78, 'Popcorn': 0.89, 'Muffin': 0.84},
    "Stationery": {
        'Pen': 0.87, 'Adhesive tape': 0.67, 'Stapler': 0.67,
        'Ruler': 0.81, 'Calculator': 0.58, 'Box': 0.96, 'Clock': 0.67
    },
    "Toys": {'Ball': 0.75, 'Flying disc': 0.46, 'Teddy bear': 0.87},
    "Tableware": {
        'Orange': 0.83, 'Plate': 0.64, 'Knife': 0.46, 'Spoon': 0.46, 'Saucer': 0.46, 'Chopsticks': 0.58
    }
}

In [ ]:
# def classify_objects(objects_list):
#     for object_info in objects_list:
#         class_name = object_info["class_name"]
#         object_id = object_info["object_id"]
#         segmentation = object_info["segmentation"]
#         box = object_info["box"]

#         scores = {
#             category: score_dict[category].get(class_name, 0) * 100
#             for category in score_dict
#         }

#         if all(score == 0 for score in scores.values()):
#             scores["Misc"] = 100

#         print(f"Class Name: {class_name}")
#         print(f"Object ID: {object_id}")
#         print(f"Box: {box}")
#         print(f"Segmentation: {segmentation}")
#         for category, score in scores.items():
#             if score != 0:
#                 print(f"{category}: {score}%")
#         print("\n")

#     print("---------------------------------------------------")

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set the testing threshold for this model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # You are now detecting only 2 classes
cfg.MODEL.ROI_HEADS.CLASS_NAMES = ['Mobile phone', 'Stapler']
cfg.MODEL.ROI_HEADS.CLASS_SELECT = [42, 70]  # indices of the classes you want to predict


In [ ]:
predictor = DefaultPredictor(cfg)


[08/23 19:15:50 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/faster_rcnn/Weights_1/model_0077999.pth ...


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


In [ ]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

       video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;

      }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

       return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label):
  data = eval_js('stream_frame("{}")'.format(label))
  return data

In [ ]:
# start streaming video from webcam--> Tries to emulate a live action implementation

from IPython.display import clear_output
from base64 import b64encode

def video_frame(label, img_data):
  data = eval_js('stream_frame("{}", "{}")'.format(label, img_data))
  return data

def img_to_js(img):
  retval, buffer = cv2.imencode('.jpg', img)
  jpg_as_text = b64encode(buffer).decode('utf-8')
  return "data:image/jpg;base64,{}".format(jpg_as_text)

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty

while True:
  js_reply = video_frame(label_html, "")
  if not js_reply:
    break

  # convert JS response to OpenCV Image
  img = js_to_image(js_reply["img"])

  outputs = predictor(img)
  objects_list = generate_objects_list(outputs, cfg)
  classify_objects(objects_list)

  v = Visualizer(img[:, :, ::-1],
                 metadata=metadata,
                 scale=1.0)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

  # Convert the output image to a format suitable for JavaScript
  js_img = img_to_js(out.get_image()[:, :, ::-1])

  # Display the output image in the HTML video element
  js_reply = video_frame(label_html, js_img)

<IPython.core.display.Javascript object>

Class Name: Apple
Object ID: 0
Box: [326.6690979003906, 419.1406555175781, 388.7960510253906, 443.6871337890625]
Fruit: 98.0%
Snacks: 69.0%


---------------------------------------------------
Class Name: Apple
Object ID: 0
Box: [45.05638122558594, 6.103491306304932, 299.6103820800781, 237.1275177001953]
Fruit: 98.0%
Snacks: 69.0%


Class Name: Apple
Object ID: 1
Box: [334.1905517578125, 371.5423278808594, 440.2726135253906, 410.4306945800781]
Fruit: 98.0%
Snacks: 69.0%


Class Name: Apple
Object ID: 2
Box: [305.9875793457031, 464.0142822265625, 374.7655334472656, 480.0]
Fruit: 98.0%
Snacks: 69.0%


---------------------------------------------------
Class Name: Apple
Object ID: 0
Box: [634.8923950195312, 0.007488441653549671, 640.0, 15.734963417053223]
Fruit: 98.0%
Snacks: 69.0%


Class Name: Apple
Object ID: 1
Box: [113.34715270996094, 463.9210205078125, 177.46697998046875, 480.0]
Fruit: 98.0%
Snacks: 69.0%


Class Name: Apple
Object ID: 2
Box: [0.0, 0.0632835403084755, 6.4836602210

KeyboardInterrupt: ignored